In [1]:
# default_exp datapipe
from nbdev import *

# Database - functions for data back-end / manipulations

This is using an alternate approach:
  - I have exported all of my Apple healthfit data from the Health app to export.zip 
  and then converted this to a SQLite database using `healthfit-to-sqlite`
  - I am then creating a "published" version of this SQLite database using
  Datasette.io
  - I have a local version of this database running at http://localhost:8081/healthkit and
  similarly I have an externally deployed version at https://my-healthkit-data.fly.dev
  - I will then run queries against this database to build the cache file (or possibly a smaller custom sqlite file)

## TODO
* This is still a work in progress
* Need to write the queries to marshall the data for each of the workouts within each group of walks
* Then cache this data - maybe try another (small) sqlite db for the caching (instead of feather)
* **NOTE: It looks like the queries are being truncated at 1000 values - need to fix**


In [100]:
#export
import os
import pandas as pd
import activityio as aio
from dateutil.parser import parse
import datetime as dt
import sqlite3 as sql
from pathlib import Path
import tomli
import klib
import subprocess
import pendulum
from sqlite_utils import Database
import reverse_geocoder as rg



In [3]:
try:
    from emmaus_walking.core import in_notebook, get_project_root, get_project_root_alternate
except:
    from core import in_notebook, get_project_root, get_project_root_alternate

In Jupyter notebook
Project root directory: /Users/mjboothaus/code/github/mjboothaus/emmaus_walking


[https://databooth.slite.com/api/s/note/MEQPRcLKFqUE6ko2vVp3Yr/Datasette-io-approach](https://databooth.slite.com/api/s/note/MEQPRcLKFqUE6ko2vVp3Yr/Datasette-io-approach) (Slite page)

e.g. start_datasette = subprocess.Popen('datasette ' + db_name + ' -m aus-covid-datasette-meta.json', stdout=subprocess.PIPE, shell=True)

In [4]:
HEALTHKIT_DATA_PATH = Path("/Users/mjboothaus/data/healthkit")
export_zip = HEALTHKIT_DATA_PATH / "export.zip"
#EXPORT_ZIP_PATH = Path("/Users/mjboothaus/data/healthkit/export.zip")

In [5]:
def convert_healthkit_export_to_sqlite():
    zip_file = export_zip.as_posix()
    zip_file_date = pendulum.instance(dt.datetime.fromtimestamp(export_zip.stat().st_ctime))

    db_file = zip_file.replace("export.zip", "healthkit.db")
    if Path(db_file).exists == True:
        Path(db_file).unlink()
    sp_cmd = "pipx run healthkit-to-sqlite " + zip_file + " " + db_file
    print(sp_cmd)
    print('---------------------------------------------------------------------------------------------')
    print('Please wait: converting healthkit export.zip to sqlite database (takes just over a minute)...')

    sp = subprocess.Popen(sp_cmd, stdout=subprocess.PIPE, shell=True)
    (sp_output, sp_err) = sp.communicate()  

    #This makes the wait possible
    sp_status = sp.wait()

    db_file_with_date = db_file.replace(".db", "_" + zip_file_date.to_date_string().replace("-", "_") + ".db")
    
    export_zip.rename(zip_file.replace(".zip", "_" + zip_file_date.to_date_string().replace("-", "_") + ".zip"))
    Path(db_file).rename(db_file_with_date)

    return db_file_with_date, sp_output

In [6]:
db_file, output = convert_healthkit_export_to_sqlite()

pipx run healthkit-to-sqlite /Users/mjboothaus/data/healthkit/export.zip /Users/mjboothaus/data/healthkit/healthkit.db
---------------------------------------------------------------------------------------------
Please wait: converting healthkit export.zip to sqlite database (takes just over a minute)...


In [7]:
output

b'Importing from HealthKit\n'

## Look at the HealthKit database via Datasette.io (and try an reduce to just needed data)

In [95]:
print("datasette " + db_file + " --setting sql_time_limit_ms 5000 &")

datasette /Users/mjboothaus/data/healthkit/healthkit_2021_07_31.db --setting sql_time_limit_ms 5000 &


In [51]:
def create_df_from_sql_query_in_file(filename_dot_sql, conn, parse_dates):
# Read the sql file
    query_file = get_project_root_alternate() / "sql" / filename_dot_sql

    with open(query_file, 'r') as query:
        # connection == the connection to your database
        sql_text = query.read()
        print(sql_text)
        df = pd.read_sql_query(sql_text, conn, parse_dates=parse_dates)
    return df

In [10]:
db = Database(db_file)

In [60]:
workouts_df = create_df_from_sql_query_in_file("select_star_walking_workouts.sql", db.conn, ['startDate', 'endDate'])

select
    id,
    duration,
    totaldistance,
    totalenergyburned,
    sourcename,
    sourceversion,
    startdate,
    enddate,
    metadata_hkweathertemperature,
    metadata_hkweatherhumidity,
    metadata_hkelevationascended,
    metadata_hkaveragemets
from
    workouts
where workoutactivitytype = "HKWorkoutActivityTypeWalking" order by id


 /* Excluded fields:
    workoutactivitytype,   # just walking
    durationunit,          # fixed - min
    totaldistanceunit,     # fixed - km
    totalenergyburnedunit, # fixed - kJ
    device,
    creationdate,          # not really of interest (start date instead)
    workout_events,        # think this is redundant info (need to check - JSON?)
    metadata_hkgroupfitness,
    metadata_hkworkoutbrandname,
    metadata_hktimezone,
    metadata_hkcoachedworkout,
    metadata_hkwasuserentered,
    metadata_hkindoorworkout,
    metadata_hkelevationascended,
    metadata_hkswimminglocationtype,
    metadata_hkaveragemets,
    metadata_healt

In [61]:
workouts_df.head()

,id,duration,totalDistance,totalEnergyBurned,sourceName,sourceVersion,startDate,endDate,metadata_HKWeatherTemperature,metadata_HKWeatherHumidity,metadata_HKElevationAscended,metadata_HKAverageMETs
0,000d7757f079ab0f75a4db50621f052894aa33c2,30.05313618381818,1.177883369302785,639.2089883072423,Michael and Ai Leen’s Apple Watch,7.1,2021-01-06 13:53:19+10:00,2021-01-06 14:23:22+10:00,73.4 degF,6900 %,3899 cm,3.82314 kcal/hr·kg
1,0018d73a8cf8c051befbc9bfb427320431b54628,97.57144786715507,5.969473837612362,1843.191264816394,Michael and Ai Leen’s Apple Watch,6.1.1,2020-03-23 09:15:24+10:00,2020-03-23 10:52:59+10:00,68 degF,6400 %,9430 cm,3.30812 kcal/hr·kg
2,00645c031954e799f2bad6ca9f26529a676bc660,91.34290488362312,5.020199223674742,2235.856794424977,Michael and Ai Leen’s Apple Watch,5.1.3,2019-04-08 11:41:03+10:00,2019-04-08 13:12:24+10:00,84 degF,4300 %,13916 cm,None
3,00e4a61ddc04be5ec7fca6130b97fb3694d07c7a,75.63339018424352,4.172883323542221,1718.733819123089,Michael and Ai Leen’s Apple Watch,6.2.1,2020-05-06 16:23:43+10:00,2020-05-06 17:39:21+10:00,71 degF,4500 %,7325 cm,3.94665 kcal/hr·kg
4,02b32c6480081df19693e7f5661f5f3c7ac65474,105.3766344666481,6.767214865892154,2543.764525288699,Michael and Ai Leen’s Apple Watch,6.2.6,2020-06-24 15:10:00+10:00,2020-06-24 17:58:14+10:00,65 degF,4900 %,8025 cm,None


In [96]:
# Trying to find the start point in each walk workout -- as date is not a date field in db not clear if sort by date and limit 1 will work
# might need to import table to pandas convert types and then export to db before doing query. Else use sqlite_utils to change column types.

start_point_df = create_df_from_sql_query_in_file("select_start_point_workout.sql", db.conn, ['date'])

select
    date,
    latitude,
    longitude,
    altitude,
    workout_id
from
    workout_points
group by
    workout_id
order by
    date desc



In [97]:
start_point_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562 entries, 0 to 561
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   date        562 non-null    datetime64[ns, UTC]
 1   latitude    562 non-null    float64            
 2   longitude   562 non-null    float64            
 3   altitude    562 non-null    float64            
 4   workout_id  562 non-null    object             
dtypes: datetime64[ns, UTC](1), float64(3), object(1)
memory usage: 22.1+ KB


In [121]:
start_point_df.head()

,date,latitude,longitude,altitude,workout_id,location,dateExcel
0,2021-07-29 20:26:14+00:00,-33.847070,151.152657,24.360422,713b1dff505e67abc656493cfa63c689458aac48,"[Canada Bay, New South Wales, AU]",2021-07-29 20:26:14
1,2021-07-28 22:45:24+00:00,-33.847106,151.152939,8.810939,c76a3094123013f7cc03faea0fb696f1ae3b69ce,"[Canada Bay, New South Wales, AU]",2021-07-28 22:45:24
2,2021-07-28 05:27:14+00:00,-33.847142,151.152727,26.479507,09c70a1e57a1fb2111c7dfc5afcbb4774f0e11fe,"[Canada Bay, New South Wales, AU]",2021-07-28 05:27:14
3,2021-07-27 21:47:09+00:00,-33.847187,151.152798,47.524326,cbb3db7719584fe7291d794ed7bd56dea4c003bf,"[Canada Bay, New South Wales, AU]",2021-07-27 21:47:09
4,2021-07-27 05:25:56+00:00,-33.847177,151.152536,27.812546,b4c63ad61db995ae240368175e57df4798337bd7,"[Canada Bay, New South Wales, AU]",2021-07-27 05:25:56


In [114]:
def get_starting_location(latitude, longitude):
    location = rg.search((latitude, longitude))
    return [location[0]['name'], location[0]['admin1'], location[0]['cc']]

In [115]:
start_point_df['location'] = start_point_df.apply(lambda row: get_starting_location(float(row[1]), float(row[2])), axis=1)

In [124]:
start_point_df['date'] = start_point_df['date'].apply(lambda dt: pendulum.instance(dt).to_datetime_string())

In [125]:
start_point_df.to_excel('start_point_df.xlsx', index=False)

In [131]:
start_point_df['workout_id'].nunique() / len(start_point_df)

1.0

In [62]:
workouts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509 entries, 0 to 508
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype                                
---  ------                         --------------  -----                                
 0   id                             509 non-null    object                               
 1   duration                       509 non-null    object                               
 2   totalDistance                  509 non-null    object                               
 3   totalEnergyBurned              509 non-null    object                               
 4   sourceName                     509 non-null    object                               
 5   sourceVersion                  509 non-null    object                               
 6   startDate                      509 non-null    datetime64[ns, pytz.FixedOffset(600)]
 7   endDate                        509 non-null    datetime64[ns, pytz.FixedOffset(6

In [63]:
workouts_df_cleaned = klib.data_cleaning(workouts_df)

Long column names detected (>25 characters). Consider renaming the following columns ['metadata_hkweather_temperature', 'metadata_hkweather_humidity', 'metadata_hkelevation_ascended'].
Shape of cleaned data: (509, 12)Remaining NAs: 450

Changes:
Dropped rows: 0
     of which 0 duplicates. (Rows: [])
Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: -0.01 MB (--20.0%)



/opt/homebrew/Caskroom/mambaforge/base/envs/emmaus_walking_py38/lib/python3.9/site-packages/klib/clean.py:76: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns.str.replace("\n", "_")


In [64]:
workouts_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 509 entries, 0 to 508
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype                                
---  ------                          --------------  -----                                
 0   id                              509 non-null    string                               
 1   duration                        509 non-null    string                               
 2   total_distance                  509 non-null    string                               
 3   total_energy_burned             509 non-null    string                               
 4   source_name                     509 non-null    category                             
 5   source_version                  509 non-null    string                               
 6   start_date                      509 non-null    datetime64[ns, pytz.FixedOffset(600)]
 7   end_date                        509 non-null    datetime64[ns, pytz.Fix

In [65]:
workouts_df_cleaned.head()

,id,duration,total_distance,total_energy_burned,source_name,source_version,start_date,end_date,metadata_hkweather_temperature,metadata_hkweather_humidity,metadata_hkelevation_ascended,metadata_hkaverage_mets
0,000d7757f079ab0f75a4db50621f052894aa33c2,30.05313618381818,1.177883369302785,639.2089883072423,Michael and Ai Leen’s Apple Watch,7.1,2021-01-06 13:53:19+10:00,2021-01-06 14:23:22+10:00,73.4 degF,6900 %,3899 cm,3.82314 kcal/hr·kg
1,0018d73a8cf8c051befbc9bfb427320431b54628,97.57144786715507,5.969473837612362,1843.191264816394,Michael and Ai Leen’s Apple Watch,6.1.1,2020-03-23 09:15:24+10:00,2020-03-23 10:52:59+10:00,68 degF,6400 %,9430 cm,3.30812 kcal/hr·kg
2,00645c031954e799f2bad6ca9f26529a676bc660,91.34290488362312,5.020199223674742,2235.856794424977,Michael and Ai Leen’s Apple Watch,5.1.3,2019-04-08 11:41:03+10:00,2019-04-08 13:12:24+10:00,84 degF,4300 %,13916 cm,<NA>
3,00e4a61ddc04be5ec7fca6130b97fb3694d07c7a,75.63339018424352,4.172883323542221,1718.733819123089,Michael and Ai Leen’s Apple Watch,6.2.1,2020-05-06 16:23:43+10:00,2020-05-06 17:39:21+10:00,71 degF,4500 %,7325 cm,3.94665 kcal/hr·kg
4,02b32c6480081df19693e7f5661f5f3c7ac65474,105.3766344666481,6.767214865892154,2543.764525288699,Michael and Ai Leen’s Apple Watch,6.2.6,2020-06-24 15:10:00+10:00,2020-06-24 17:58:14+10:00,65 degF,4900 %,8025 cm,<NA>


In [66]:
db_file

'/Users/mjboothaus/data/healthkit/healthkit_2021_07_31.db'

In [67]:
workouts_csv = HEALTHKIT_DATA_PATH / "workouts.csv"

In [68]:
workouts_df_cleaned.to_csv(workouts_csv, index=False)

In [69]:
workouts_df_cleaned.columns

Index(['id', 'duration', 'total_distance', 'total_energy_burned',
       'source_name', 'source_version', 'start_date', 'end_date',
       'metadata_hkweather_temperature', 'metadata_hkweather_humidity',
       'metadata_hkelevation_ascended', 'metadata_hkaverage_mets'],
      dtype='object')

In [70]:
workouts_df_cleaned.nunique()

id                                509
duration                          508
total_distance                    508
total_energy_burned               508
source_name                         2
source_version                     28
start_date                        507
end_date                          507
metadata_hkweather_temperature     66
metadata_hkweather_humidity        83
metadata_hkelevation_ascended     366
metadata_hkaverage_mets           293
dtype: int64

In [48]:
[col for col in workouts_df.columns.values if col not in workouts_df_cleaned.columns.values]

['totalDistance',
 'totalEnergyBurned',
 'sourceName',
 'sourceVersion',
 'startDate',
 'endDate',
 'metadata_HKWeatherTemperature',
 'metadata_HKWeatherHumidity',
 'metadata_HKElevationAscended',
 'metadata_HKAverageMETs']

In [49]:
tables_df = create_df_from_sql_query_in_file("list_all_tables.sql", db.conn)

SELECT name FROM sqlite_master WHERE type = "table"


In [50]:
tables_df

,name
0,rHeight
1,rBodyMass
2,rHeartRate
3,rBloodPressureSystolic
4,rBloodPressureDiastolic
5,rStepCount
6,rDistanceWalkingRunning
7,rBasalEnergyBurned
8,rActiveEnergyBurned
9,rFlightsClimbed


### Looking at sqlite version of the cached data derived from individual walk files in walk groups

In [ ]:
db_file = Path('emmaus_walking.db')

In [ ]:
!echo "datasette" $db_file "&"

In [ ]:
LOCAL_DB_URL = 'http://localhost:8081/'
HOSTED_DB_URL = 'https://my-healthkit-data.fly.dev/'

In [ ]:
url_CSV = 'http://localhost:8081/healthkit.csv?sql=select%0D%0A++id%2C%0D%0A++workoutActivityType%2C%0D%0A++duration%2C%0D%0A++durationUnit%2C%0D%0A++totalDistance%2C%0D%0A++totalDistanceUnit%2C%0D%0A++totalEnergyBurned%2C%0D%0A++totalEnergyBurnedUnit%2C%0D%0A++sourceName%2C%0D%0A++sourceVersion%2C%0D%0A++creationDate%2C%0D%0A++startDate%2C%0D%0A++endDate%2C%0D%0A++metadata_HKTimeZone%2C%0D%0A++metadata_HKWeatherTemperature%2C%0D%0A++metadata_HKWeatherHumidity%2C%0D%0A++device%2C%0D%0A++metadata_HKElevationAscended%2C%0D%0A++metadata_HKAverageMETs%0D%0Afrom%0D%0A++workouts%0D%0Aorder+by%0D%0A++id%0D%0Alimit%0D%0A++101'

In [ ]:
url_CSV2 = 'http://localhost:8081/healthkit.csv?sql=select%0D%0A++id%2C%0D%0A++workoutActivityType%2C%0D%0A++duration%2C%0D%0A++durationUnit%2C%0D%0A++totalDistance%2C%0D%0A++totalDistanceUnit%2C%0D%0A++totalEnergyBurned%2C%0D%0A++totalEnergyBurnedUnit%2C%0D%0A++sourceName%2C%0D%0A++sourceVersion%2C%0D%0A++creationDate%2C%0D%0A++startDate%2C%0D%0A++endDate%2C%0D%0A++metadata_HKTimeZone%2C%0D%0A++workout_events%2C%0D%0A++metadata_HKWeatherTemperature%2C%0D%0A++metadata_HKWeatherHumidity%2C%0D%0A++device%2C%0D%0A++metadata_HKElevationAscended%2C%0D%0A++metadata_HKAverageMETs%2C%0D%0A++metadata_HKMaximumSpeed%2C%0D%0A++metadata_HKAverageSpeed%0D%0Afrom%0D%0A++workouts%0D%0Awhere%0D%0A++workoutActivityType+in+%28%3Ap0%2C+%3Ap1%29%0D%0Aorder+by%0D%0A++creationDate%0D%0Alimit%0D%0A++101&p0=HKWorkoutActivityTypeWalking&p1=HKWorkoutActivityTypeHiking&_size=max'

In [ ]:
workouts_df = pd.read_csv(url_CSV)

#print((LOCAL_DB_URL + 'workout.json'))
#workout_df = pd.read_json(LOCAL_DB_URL + 'workouts.json')

In [ ]:
workouts_df.nunique()

In [ ]:
workouts_df.info()

In [ ]:
workouts_clean_df = klib.data_cleaning(workouts_df)

In [ ]:
workouts_clean_df.info()

In [ ]:
workouts_df.head()

In [ ]:
workouts_fly_df = pd.read_csv(url_CSV.replace(LOCAL_DB_URL, HOSTED_DB_URL))

In [ ]:
workouts_fly_df.head()

In [ ]:
workout_points_SQL = 'http://localhost:8081/healthkit.csv?sql=select%0D%0A++rowid%2C%0D%0A++date%2C%0D%0A++latitude%2C%0D%0A++longitude%2C%0D%0A++altitude%2C%0D%0A++speed%0D%0Afrom%0D%0A++workout_points%0D%0Awhere%0D%0A++workout_id+%3D+%22'
workout_id = 'a34036ff616122952fa67c9bc11a493f8642dd7c' + '%22'

workout_points_df = pd.read_csv(workout_points_SQL + workout_id, parse_dates=True)

In [ ]:
workout_points_df.head()

In [ ]:
workout_points_df.info()

In [ ]:
WALK_DETAILS_FILE = 'walk_details.toml'
walk_details = Path('../' + WALK_DETAILS_FILE)


In [ ]:
walk_details

In [ ]:
with open(walk_details, encoding="utf-8") as f:
    walk_details_dict = tomli.load(f)

In [ ]:
walk_details_dict

In [ ]:
pd.DataFrame(walk_details_dict, )

In [ ]:
#export
def calc_walk_stats(walk_data):
    total_time = dt.timedelta(0)
    total_distance = 0

    for iHike, hike in enumerate(walk_data):
        total_time += hike.index.max()
        # print(iHike+1, walk_date[iHike], hike.index.max(), hike['dist'].max() / 1e3)
        total_distance += hike['dist'].max()
    total_distance /= 1e3

    start_coord = walk_data[0][['lat', 'lon']].iloc[0].tolist()
    end_coord = walk_data[-1][['lat', 'lon']].iloc[-1].tolist()
    return total_time, total_distance, start_coord, end_coord


# TODO: use st.cache() and also look to pre-load and cache/feather data (or similar) - NB: use of @st.cache() below didn't work
def load_and_cache_raw_walk_data(walk_name, sample_freq, conn):
    RAW_FIT_FILE_PATH = 'icloud/Data/HealthFit/FIT' 
    fit_dir = Path.home()/RAW_FIT_FILE_PATH
    data_dir = fit_dir/walk_name[0:3]
    print(data_dir.ls())
    data_files = [file for file in os.listdir(data_dir) if file.endswith('.fit')]
    walk_files = sorted(data_files)
    print(walk_files)

    walk_data = []
    walk_date = []

    for iFile, file in enumerate(walk_files):
        walk_df = pd.DataFrame(aio.read(data_dir + file))
        walk_data.append(walk_df)
        walk_date.append(parse(file[0:17]))
        walk_df['WalkName'] = walk_name
        walk_df['WalkNumber'] = iFile
        walk_df[['alt', 'dist', 'lat', 'lon', 'speed', 'WalkName', 'WalkNumber']].to_sql('walks', conn, if_exists='append', index=False)
               
    total_time, total_distance, start_coord, end_coord = calc_walk_stats(walk_data)
    walk_stats = [total_time, total_distance, start_coord, end_coord]
    #print(start_coord)
    walk_merged = pd.concat(walk_data)
    points = walk_merged[['lat', 'lon']].values.tolist()
    points = [tuple(point) for ipoint, point in enumerate(points) if ipoint % sample_freq == 0]
    return walk_data, walk_date, walk_files, points, walk_stats

In [ ]:
def create_walk_cached_data_for_app(db_file, n_rows_used=5):
    # read in all of the walks data and sample at an appropriate frequency and cache for faster use in the app
    db_conn = sql.connect(db_file)
    walk_df = pd.read_sql_query('SELECT * FROM walks', db_conn)

    UNUSED_COLUMNS = ['dist', 'speed']

    walk_df.drop(UNUSED_COLUMNS, axis=1, inplace=True)
    walk_df.dropna(inplace=True)      # TODO: Check why there are a few NaNs
    walk_df = walk_df.iloc[::n_rows_used].reset_index()    # downsample

    walk_df.to_feather(Path(db_file.as_posix().replace('.db', '.cache.feather')))
    
    return walk_df

In [ ]:
# Not working yet -- this is the alternate approach to using the individual .FIT files
# walk_df = create_walk_cached_data_for_app(db_file, 10)

In [ ]:
# walk_df[walk_df['lat'].isna()]

In [ ]:
Path(db_file.as_posix().replace('.db', '.cache.feather'))

In [ ]:
# walk_df = pd.read_feather(Path(db_file.as_posix().replace('.db', '.cache.feather')))

In [ ]:
# walk_df.info()

In [ ]:
# walk_df['WalkName'].unique()